# Import the packages

In [4]:
#conda install scikit-learn=1.0.1.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics, model_selection 
from sklearn.linear_model import LogisticRegression
#from sklearn.calibration import CalibrationDisplay
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression 
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, r2_score
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.24.2.


# Data Import

In [6]:
df = pd.read_csv('../data/processed_chinese_mnist_part_1.csv').append(
    pd.read_csv('../data/processed_chinese_mnist_part_2.csv')).reset_index(drop = True)

In [7]:
df.head(5)

,label,cn_label,value,0,1,2,3,4,5,6,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,1,零,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10,九,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11,十,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12,百,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13,千,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Make it binary

In [8]:
df.label = df.label.apply(lambda x: 1 if x == 1 else 0)

# Train Test Spilt

In [13]:
X, y = df.iloc[:,3:], df.iloc[:,0]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2)

# Use 4 binary classifers

In [15]:
#constructing 4 classifiers
sgd_clf = SGDClassifier(random_state = 42,loss = 'modified_huber')
model1 = sgd_clf.fit(Xtrain,ytrain)
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42, probability=True)

#Fitting 4 models on training set
model1 = sgd_clf.fit(Xtrain, ytrain)
model2 = log_clf.fit(Xtrain, ytrain)
model3 = rnd_clf.fit(Xtrain, ytrain)
model4 = svm_clf.fit(Xtrain, ytrain)

#Implemeting Bagging to reduce bias for first classifier
bag_clf1 = BaggingClassifier(SGDClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1)
model5 = bag_clf1.fit(Xtrain, ytrain)
y_pred1 = bag_clf1.predict(Xtest)


#Vote one time for classifiers w/o bagging
voting_clf1 = VotingClassifier(
estimators=[('sgd',sgd_clf),('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
voting='soft')
final_model1 = voting_clf1.fit(Xtrain, ytrain)



In [17]:
y_pred1

array([0, 0, 0, ..., 0, 0, 0])

# Voting among the four binary classifers: SGD, Logistic Regression, Random Forest, and SVM



In [22]:
voting_clf1.score(Xtrain, ytrain)

0.9979166666666667

In [23]:
voting_clf1.score(Xtest, ytest)

0.9583333333333334

# Bagging : SGD classifer

In [19]:
bag_clf1.score(Xtrain, ytrain)

0.9325

In [21]:
bag_clf1.score(Xtest, ytest)

0.9366666666666666

In [25]:
bag_clf2 = BaggingClassifier(LogisticRegression(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1)
model6 = bag_clf2.fit(Xtrain, ytrain)
y_pred2 = bag_clf2.predict(Xtest)

## Bagging: Logistic Regression makes a pretty good predictions

In [26]:
model6.score(Xtrain, ytrain)

0.9325

In [29]:
model6.score(Xtest, ytest)

0.9366666666666666

# Try more bagging here

In [27]:
bag_clf3 = BaggingClassifier(RandomForestClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1)
model7 = bag_clf3.fit(Xtrain, ytrain)
y_pred3 = bag_clf3.predict(Xtest)

In [28]:
bag_clf4 = BaggingClassifier(SVC(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1)
model8 = bag_clf4.fit(Xtrain, ytrain)
y_pred4 = bag_clf4.predict(Xtest)

In [32]:
model7.score(Xtrain, ytrain)

0.9325

In [33]:
model7.score(Xtest, ytest)

0.9366666666666666

In [34]:
model8.score(Xtrain, ytrain)

0.9325

In [35]:
model8.score(Xtest, ytest)

0.9366666666666666